In [1]:
import pandas as pd
import numpy as np

import requests
import datetime

import yfinance as yf

from alpaca.trading.client import TradingClient
from alpaca.trading.enums import OrderSide, TimeInForce
from alpaca.trading.requests import MarketOrderRequest, StopLossRequest
from alpaca.trading.stream import TradingStream

import config
from reversal_bot import Reversal
from portfolio import Portfolio

In [2]:
def read_tickers(filename):
    tickers_list = []
    with open(filename, 'r') as file:
        for line in file:
            ticker = line.strip()
            tickers_list.append(ticker)
        return tickers_list

In [3]:
# (deprecated) list of tickers on the NASDAQ 100
tickers = read_tickers('nasdaq_100.txt')

In [4]:
bot = Reversal(tickers)

In [5]:
bot.push_to_watchlist()

NDX: Period '305m' is invalid, must be one of ['1mo', '3mo', '6mo', 'ytd', '1y', '2y', '5y', '10y', 'max']
NDX: Period '305m' is invalid, must be one of ['1mo', '3mo', '6mo', 'ytd', '1y', '2y', '5y', '10y', 'max']


In [6]:
bot.watchlist

['AAPL',
 'ADI',
 'AKAM',
 'AMAT',
 'AMGN',
 'ATVI',
 'AVGO',
 'BIIB',
 'BMRN',
 'CHKP',
 'CHTR',
 'COST',
 'CSCO',
 'DLTR',
 'EA',
 'FAST',
 'GILD',
 'INCY',
 'INTC',
 'ISRG',
 'LRCX',
 'MAR',
 'MCHP',
 'MDLZ',
 'MNST',
 'MSFT',
 'NCLH',
 'NTAP',
 'NXPI',
 'ORLY',
 'PAYX',
 'PCAR',
 'QCOM',
 'REGN',
 'ROST',
 'SBUX',
 'STX',
 'SWKS',
 'TSCO',
 'TXN',
 'ULTA',
 'VOD',
 'VRTX',
 'WDC',
 'XRAY']

In [7]:
buys = bot.buy_query()

In [8]:
print(buys)

[]


In [13]:
client = TradingClient(api_key = config.API_KEY, secret_key=config.SECRET_KEY, paper=True)
account = dict(client.get_account())
for i,j in account.items():
    print(i, j)

id 5eb25c71-0f3d-41c6-bb4e-01c08c09f8ae
account_number PA31PUYR39KM
status AccountStatus.ACTIVE
crypto_status AccountStatus.ACTIVE
currency USD
buying_power 200000
regt_buying_power 200000
daytrading_buying_power 0
non_marginable_buying_power 100000
cash 100000
accrued_fees 0
pending_transfer_out None
pending_transfer_in 0
portfolio_value 100000
pattern_day_trader False
trading_blocked False
transfers_blocked False
account_blocked False
created_at 2023-07-27 04:59:34.716957+00:00
trade_suspended_by_user False
multiplier 2
shorting_enabled True
equity 100000
last_equity 100000
long_market_value 0
short_market_value 0
initial_margin 0
maintenance_margin 0
last_maintenance_margin 0
sma 100000
daytrade_count 0


In [14]:
client.close_position('APPL')

APIError: {"code":40010001,"message":"could not find asset for \"APPL\""}

In [15]:
datetime.datetime.now()

datetime.datetime(2023, 8, 2, 19, 33, 9, 540369)

In [16]:
datetime.datetime.now()[3]

TypeError: 'datetime.datetime' object is not subscriptable